##### データセットの場所やバッチサイズなどの定数値の設定

In [ ]:
import os
os.environ['CUDA_DEVICE_ORDER'] = 'PCI_BUS_ID'


# 使用するデバイス
# GPU を使用しない環境（CPU環境）で実行する場合は DEVICE = 'cpu' とする
DEVICE = 'cuda:0'

# 全ての訓練データを一回ずつ使用することを「1エポック」として，何エポック分学習するか
N_EPOCHS = 50

# 学習時のバッチサイズ
BATCH_SIZE = 25

# 訓練データセット（画像ファイルリスト）のファイル名
TRAIN_DATASET_CSV = './tinySTL10/train_list.csv'

# テストデータセット（画像ファイルリスト）のファイル名
TEST_DATASET_CSV = './tinySTL10/test_list.csv'

# 画像ファイルの先頭に付加する文字列（データセットが存在するディレクトリのパス）
DATA_DIR = './tinySTL10'

# 画像サイズ（実際には，ニューラルネットワーク内部の前処理にて 224x224 にリサイズしている）
H = 96 # 縦幅
W = 96 # 横幅
C = 3 # チャンネル数（カラー画像なら3，グレースケール画像なら1）

# 学習結果の保存先フォルダ
MODEL_DIR = './ViT_models'

# 学習結果のニューラルネットワークの保存先
MODEL_FILE = os.path.join(MODEL_DIR, 'ViT_object_recognizer_model.pth')

##### ニューラルネットワークモデルの定義

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms


# 画像のパッチ分割および各パッチの平坦化を行うクラス
# add_cls_token=True が指定されている場合は，CLSトークンを付加する
class ToFlattenedPatches(nn.Module):
    def __init__(self, patch_size, in_channels=3, add_cls_token=False):
        super(ToFlattenedPatches, self).__init__()
        self.patch_size = patch_size
        if add_cls_token:
            self.cls = nn.Parameter(torch.zeros(1, 1, in_channels*(patch_size**2)))
        else:
            self.cls = None
    def forward(self, x):
        h = torch.cat([
            torch.cat([
                torch.reshape(q, (len(q), 1, -1)) for q in torch.split(p, self.patch_size, dim=3)
            ], dim=1) for p in torch.split(x, self.patch_size, dim=2)
        ], dim=1) # パッチ分割 + 各パッチ平坦化
        if self.cls is not None:
            h = torch.cat([self.cls.repeat_interleave(x.size()[0], dim=0), h], dim=1) # CLSトークン付加
        return h


# 位置エンコーディングを行うクラス
class PositionEmbeddings(nn.Module):
    def __init__(self, embed_dim):
        super(PositionEmbeddings, self).__init__()
        self.dim = embed_dim
    def forward(self, t):
        d = self.dim // 2
        e = torch.log(torch.tensor(10000, device=t.device)) / d
        e = torch.exp(torch.arange(d, device=t.device) * -e)
        e = torch.unsqueeze(t, dim=-1) * e.repeat([1 for i in t.size()])
        return torch.cat((e.sin(), e.cos()), dim=-1)


# Vision Transformer用の単位ブロック
class ViTBlock(nn.Module):
    def __init__(self, embed_dim, num_heads):
        super(ViTBlock, self).__init__()
        self.ln1 = nn.LayerNorm([embed_dim])
        self.ln2 = nn.LayerNorm([embed_dim])
        self.mha = nn.MultiheadAttention(embed_dim=embed_dim, num_heads=num_heads, batch_first=True)
        self.mlp = nn.Sequential(
            nn.Linear(in_features=embed_dim, out_features=embed_dim),
            nn.ReLU(),
            nn.Linear(in_features=embed_dim, out_features=embed_dim),
        )
    def forward(self, x):
        h = self.ln1(x) # Layer Normalization
        h, _ = self.mha(h, h, h) # Multihead Attention
        x = h + x # skip connection
        h = self.ln2(x) # Layer Normalization
        h = self.mlp(h) # Multi Layer Perceptron
        y = h + x # skip connection
        return y


# Vision TransformerによりSTL10物体画像認識AIを実現するニューラルネットワーク
class STL10Recognizer(nn.Module):

    # C: 入力画像のチャンネル数
    # N: 認識対象となるクラスの数
    # patch_size: 画像を正方形のパッチに分割する際のパッチサイズ（各パッチの一辺の長さ）
    # embed_dim: 各パッチを何次元のベクトル（埋め込みベクトル）で表現するか
    # num_heads: マルチヘッドアテンションにおけるヘッドの数
    def __init__(self, C, N, patch_size=16, embed_dim=128, num_heads=8):
        super(STL10Recognizer, self).__init__()

        # データ拡張
        self.data_augment = transforms.RandomApply(torch.nn.ModuleList([
            transforms.RandomHorizontalFlip(p=0.5), # 確率0.5で左右反転
            transforms.ColorJitter(brightness=0.2, contrast=0.15, saturation=0.1, hue=0.05), # カラージッター（明度を±20%，コントラストを±15%，彩度を±10%，色相を±5%の範囲内でランダムに変更）
            transforms.RandomAffine(degrees=(-15, 15), scale=(0.8, 1.2)), # -15度〜15度の範囲でランダムに回転，さらに80%〜120%の範囲内でランダムにスケーリング
            transforms.RandomErasing(p=0.5), # 確率0.5で一部を消去
        ]), p=0.5)

        # 前処理
        self.preprocess = transforms.CenterCrop(224) # 256x256 ピクセルの入力画像から中央 224x224 ピクセルを取り出す

        # パッチ分割および埋め込み
        self.patch_embed = nn.Sequential(
            ToFlattenedPatches(patch_size=patch_size, add_cls_token=True), # パッチ分割 + 平坦化
            nn.Linear(in_features=C*(patch_size**2), out_features=embed_dim), # 平坦化後の各パッチを埋め込みベクトルに変換
        )

        # 位置エンコーディング
        self.pos_embed = PositionEmbeddings(embed_dim)

        # 単位ブロック（マルチヘッドアテンション + MLP）
        self.vb1 = ViTBlock(embed_dim=embed_dim, num_heads=num_heads)
        self.vb2 = ViTBlock(embed_dim=embed_dim, num_heads=num_heads)
        self.vb3 = ViTBlock(embed_dim=embed_dim, num_heads=num_heads)
        self.vb4 = ViTBlock(embed_dim=embed_dim, num_heads=num_heads)
        self.vb5 = ViTBlock(embed_dim=embed_dim, num_heads=num_heads)
        self.vb6 = ViTBlock(embed_dim=embed_dim, num_heads=num_heads)

        # 最終層の全結合層
        self.fc = nn.Linear(in_features=embed_dim, out_features=N)

    def forward(self, x, testmode=False):
        if not testmode:
            x = self.data_augment(x) # 訓練時のみデータ拡張（テスト時は実行しない）

        # 前処理
        x = self.preprocess(x)

        # 入力画像をパッチ分割し，各パッチを埋め込みベクトルに変換
        x = self.patch_embed(x)

        # 位置エンコーディングを実行し，その情報を付加
        t = torch.arange(x.size()[1]).repeat(x.size()[0], 1).to(x.device)
        t = self.pos_embed(t)
        x = x + t

        # （マルチヘッドアテンション + MLP）× 6
        h = self.vb1(x)
        h = self.vb2(h)
        h = self.vb3(h)
        h = self.vb4(h)
        h = self.vb5(h)
        h = self.vb6(h)

        # CLSトークンに対応する情報のみを取り出す
        h = h[:, 0, :]

        # 最終層
        y = self.fc(h)
        return y

##### 訓練データセットの読み込み

In [ ]:
import pickle
from torch.utils.data import DataLoader, random_split
from mylib.data_io import CSVBasedDataset


# CSVファイルを読み込み, 訓練データセットを用意
dataset = CSVBasedDataset(
    filename = TRAIN_DATASET_CSV,
    items = [
        'File Path', # X
        'Class Name' # Y
    ],
    dtypes = [
        'image', # Xの型
        'label' # Yの型
    ],
    dirname = DATA_DIR,
    img_mode = 'color' # 強制的にカラー画像として読み込む
)
with open(os.path.join(MODEL_DIR, 'fdicts.pkl'), 'wb') as fdicts_file:
    pickle.dump(dataset.forward_dicts, fdicts_file)

# 認識対象のクラス数を取得
n_classes = len(dataset.forward_dicts[1])

# 訓練データセットを分割し，一方を検証用に回す
dataset_size = len(dataset)
valid_size = int(0.1 * dataset_size) # 全体の 3% を検証用に
train_size = dataset_size - valid_size # 残りの 97% を学習用に
train_dataset, valid_dataset = random_split(dataset, [train_size, valid_size])

# 訓練データおよび検証用データをミニバッチに分けて使用するための「データローダ」を用意
train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, pin_memory=True)
valid_dataloader = DataLoader(valid_dataset, batch_size=BATCH_SIZE, shuffle=False, pin_memory=True)

##### 学習処理の実行

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
from mylib.visualizers import LossVisualizer


# ニューラルネットワークの作成
model = STL10Recognizer(C=C, N=n_classes, patch_size=16, embed_dim=256, num_heads=8).to(DEVICE)

# 最適化アルゴリズムの指定（ここでは SGD でなく Adam を使用）
optimizer = optim.Adam(model.parameters())

# 損失関数：クロスエントロピー損失を使用
loss_func =  nn.CrossEntropyLoss()

# 損失関数値を記録する準備
loss_viz = LossVisualizer(['train loss', 'valid loss'])

# 勾配降下法による繰り返し学習
for epoch in range(N_EPOCHS):

    print('Epoch {0}:'.format(epoch + 1))

    # 学習
    model.train()
    sum_loss = 0
    for X, Y in tqdm(train_dataloader): # X, Y は CSVBasedDataset クラスの __getitem__ 関数の戻り値に対応
        for param in model.parameters():
            param.grad = None
        X = X.to(DEVICE)
        Y = Y.to(DEVICE)
        Y_pred = model(X) # 入力画像 X を現在のニューラルネットワークに入力し，出力の推定値を得る
        loss = loss_func(Y_pred, Y) # 損失関数の現在値を計算
        loss.backward() # 誤差逆伝播法により，個々のパラメータに関する損失関数の勾配（偏微分）を計算
        optimizer.step() # 勾配に沿ってパラメータの値を更新
        sum_loss += float(loss) * len(X)
    avg_loss = sum_loss / train_size
    loss_viz.add_value('train loss', avg_loss) # 訓練データに対する損失関数の値を記録
    print('train loss = {0:.6f}'.format(avg_loss))

    # 検証
    model.eval()
    sum_loss = 0
    n_failed = 0
    with torch.inference_mode():
        for X, Y in tqdm(valid_dataloader):
            X = X.to(DEVICE)
            Y = Y.to(DEVICE)
            Y_pred = model(X, testmode=True)
            loss = loss_func(Y_pred, Y)
            sum_loss += float(loss) * len(X)
            n_failed += torch.count_nonzero(torch.argmax(Y_pred, dim=1) - Y) # 推定値と正解値が一致していないデータの個数を数える
    avg_loss = sum_loss / valid_size
    loss_viz.add_value('valid loss', avg_loss) # 検証用データに対する損失関数の値を記録
    accuracy = (valid_size - n_failed) / valid_size
    print('valid loss = {0:.6f}'.format(avg_loss))
    print('accuracy = {0:.2f}%'.format(100 * accuracy))
    print('')

# 学習結果のニューラルネットワークモデルをファイルに保存
model = model.to('cpu')
torch.save(model.state_dict(), MODEL_FILE)

# 損失関数の記録をファイルに保存
loss_viz.save(v_file=os.path.join(MODEL_DIR, 'loss_graph.png'), h_file=os.path.join(MODEL_DIR, 'loss_history.csv'))

##### テストデータセットの読み込み

In [ ]:
import pickle
from torch.utils.data import DataLoader
from mylib.data_io import CSVBasedDataset


# CSVファイルを読み込み, テストデータセットを用意
with open(os.path.join(MODEL_DIR, 'fdicts.pkl'), 'rb') as fdicts_file:
    fdicts = pickle.load(fdicts_file)
test_dataset = CSVBasedDataset(
    filename = TEST_DATASET_CSV,
    items = [
        'File Path', # X
        'Class Name' # Y
    ],
    dtypes = [
        'image', # Xの型
        'label' # Yの型
    ],
    dirname = DATA_DIR,
    fdicts = fdicts,
    img_mode = 'color' # 強制的にカラー画像として読み込む
)
test_size = len(test_dataset)
rdict = test_dataset.reverse_dicts[1]

# 認識対象のクラス数を取得
n_classes = len(test_dataset.forward_dicts[1])

# テストデータをミニバッチに分けて使用するための「データローダ」を用意
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, pin_memory=True)

##### 学習済みニューラルネットワークモデルのロード

In [ ]:
import torch


# ニューラルネットワークモデルとその学習済みパラメータをファイルからロード
model = STL10Recognizer(C=C, N=n_classes, patch_size=16, embed_dim=128, num_heads=8)
model.load_state_dict(torch.load(MODEL_FILE))

##### 単一画像に対するテスト処理の実行

In [ ]:
import torch
from mylib.data_io import show_single_image


model = model.to(DEVICE)
model.eval()

# index 番目のテストデータをニューラルネットワークに入力してみる
while True:
    print('index?: ', end='')
    val = input()
    if val == 'exit': # 'exit' とタイプされたら終了
        break
    index = int(val)
    x, y = test_dataset[index]
    x = x.reshape(1, *x.size()).to(DEVICE)
    with torch.inference_mode():
        y_pred = model(x, testmode=True)
    y_pred = torch.argmax(y_pred, dim=1)
    print('')
    print('estimated:', rdict[int(y_pred)])
    print('ground truth:', rdict[int(y)])
    print('')
    show_single_image(x.to('cpu'), title='input image', sec=1)

##### 全ての画像に対するテスト処理の実行

In [ ]:
import torch
from tqdm import tqdm


model = model.to(DEVICE)
model.eval()

# テストデータセットを用いて認識精度を評価
n_failed = 0
with torch.inference_mode():
    for X, Y in tqdm(test_dataloader):
        X = X.to(DEVICE)
        Y = Y.to(DEVICE)
        Y_pred = model(X, testmode=True)
        n_failed += torch.count_nonzero(torch.argmax(Y_pred, dim=1) - Y) # 推定値と正解値が一致していないデータの個数を数える
    accuracy = (test_size - n_failed) / test_size
    print('accuracy = {0:.2f}%'.format(100 * accuracy))
    print('')